# Homework 5: Recurrent Neural Networks

#### Introduction to Natural Language Processing

* Hyerin, Seo. (hyseo@students.uni-mainz.de)
* Yeonwoo, Nam. (yeonam@students.uni-mainz.de)
* Yevin, Kim. (kyevin@students.uni-mainz.de)

In this homework we're going to look at Syntactic Constituency Parsing with Recurrent Neural Networks (RNNs).

*Total Points: 20P*

In [1]:
# For this Exercise, we need scikit-learn
# Install if missing
import sys
!{sys.executable} -m pip install scikit-learn

# Or if this doesn't work, you can install it via your terminal in your anaconda environment
# pip install scikit-learn

# Evaluation

*TASK 1: Describe the above dataset! (1P)* -> Evaluation: **XX/1**

*TASK 2: Describe how you can develop a training approach using a sequence-to-sequence approach. (2P)* -> Evaluation: **XX/2**

*TASK 3: What issues do you think can arrise from a sequence to sequence approach for this task? (1P)* -> Evaluation: **XX/1**

*TASK 4: Explain what a sequence labeling task is! (3P)* -> Evaluation: **XX/2**

*TASK 5: Write relative scale encoding for the following input! (1P)* -> Evaluation: **XX/2**

*TASK 6:  Now write a function that converts the tree to a linearized tree in an relative scale encoding. (2P)* -> Evaluation: **XX/2**

*TASK 7: Create Class Labels. Fill in sentences, encodings and labels variables. (2P)* -> Evaluation: **XX/2**

*TASK 8: Build a PyTorch Dataset where the input consists of GloVe word indices, and utilize the class indices generated in the previous task as labels. (2P)* -> Evaluation: **XX/2**

*TASK 9: Create a bidirectional RNN that initialized the embedding with the GloVe embedding! (2P)* -> Evaluation: **XX/2**

*Task 10: Now create a training loop and train on your created dataloader! Print out the loss after each epoch. (3P)* -> Evaluation: **XX/3**

*Task 11: Now, test your bidirectional RNN! Use accuracy as a metric. (1P)* -> Evaluation: **XX/1**

**Total: XX/20**

# Section 1: Syntactic Constituency Parsing

In this section, we will explore syntactic constituency parsing.

Syntactic constituency parsing: Breaking down a sentence into its basic parts (like nouns and verbs) and showing how they fit together to form the sentence's structure - Expressed in a tree structure (most of the time). It helps understand the grammatical makeup of a sentence.

**Example**

<img src="05_example2.png" alt="drawing" width="600"/>

From Wikipedia: "**Syntactic parsing** is the automatic _analysis of syntactic structure_ of natural language, especially syntactic relations (in dependency grammar) and labelling spans of constituents (in constituency grammar)."

From Speech and Language Processing. Daniel Jurafsky & James H. Martin: "The fundamental notion underlying the idea of **constituency** is that of abstraction — groups of words behaving as single units, or constituents. A significant part of developing a grammar involves discovering the inventory of constituents present in the language." 




We will use the Penn Treebank Dataset for our exercise sheet. See https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html for explanation of the tags.



In [2]:
import nltk
from nltk.corpus import treebank
    
# Download the Penn Treebank datas
treebank_trees = treebank.parsed_sents()

# Get an example
sorted_treebank_trees = sorted(treebank_trees, key=lambda tree: len(tree.leaves()))
test_example = sorted_treebank_trees[31].copy()
print(test_example)

(S (NP-SBJ (DT This)) (VP (VBZ is) (NP-PRD (NNP Japan))) (. ?))


**Task 1: Describe the above dataset! (1P)**

The Penn Treebank dataset offers a sentence structure parse tree. It identifies the sentence structure in an easy to understand way. Taking a look at the example, it splits into Subject (NP-SBJ) "This", Verb Phrase (VP) "is" Predicate (NP-PRD) Japan, and Punctuation. It also labels each word with its respective part of speech. For instance, "This" is a determiner (DT). Finally, the data set presents a tree-like structure that demonstrates the combinations of each component to create a sentence.




Your goal is to generate syntactic tree structures for English sentences.

**Task 2: Describe how you can develop a training approach for a bidirectonal RNN in order to perform Syntactic Constituency Parsing using a sequence-to-sequence approach on the above dataset. (2P)**

A bidirectional RNN for syntactic component analysis must implement a decoder network that reads the input sequence and processes it in both directions to capture contextual information and generate a parsed tree structure. 
 First, we need to perform the equivalent of data-mapping. An input sequence representing a sentence in the Penn Treebank dataset is converted to a numeric representation by tokenizing the sentence as a word, and the output sequence representing the tree structure for that sentence is encoded as a sequence of numbers.
An effective model is one that minimizes the difference between the predicted tree structure and the actual tree structure. Therefore, using a loss function such as the cross-entropy loss function, the weights should be adjusted to minimize the loss.
After that, the dataset should be validated, and the accuracy of predicting the tree structure should be used as an evaluation factor.


**Task 3: What issues do you think can arrise from a sequence to sequence approach for this task? (1P)**

 Sequence-to-sequence models work best when input and output pairs are of fixed length, so additional techniques will be needed to handle variable-length sentences. Also, because they process input sequences sequentially, they lack context about the overall sentence structure, which may not be favorable for capturing long-term dependencies. In addition, the hierarchical nature of tree structures means that a simple sequence-to-sequnce approach may not be effective.



An easier way to solve this task is to transform the task to a sequence labeling task.

**Task 4: Explain what a sequence labeling task is! (2P)**

A sequence labeling task aims to predict a label for each element of a sequence by assigning a sequence of labels to each sequence element. An example is POS tagging, which assigns a part of speech or lexical class to each word in a sentence.

To transform the task to a sequence labeling task, we do the following:

Let $w_i$ be a word located at position $i$ in the sentence, for $$1 \leq i \leq \vert w \vert −1.$$ We will assign it a 2-tuple label $$l_i = (n_i , c_i ),$$where: $n_i$ is an integer that encodes the number of common ancestors between $w_i$ and $w_{i+1}$, and $c_i$ is the nonterminal symbol at the lowest common ancestor.

Encoding:

1. **Absolute scale**: The simplest encoding is to make $n_i$ directly equal to the number of ancestors in common between $w_i$ and $w_{i+1}$.
2. **Relative scale**: A second and better variant consists in making $n_i$ represent the difference with respect to the number of ancestors encoded in $n_{i−1}$. Its main advantage is that the size of the label set is reduced considerably.

**Example:**

<img src="05_example.png" alt="drawing" width="600"/>


**Task 5: Write a linearized tree in an relative scale encoding for the following input. (2P)**

(S (NP-SBJ (DT This)) (VP (VBZ is) (NP-PRD (NNP Japan))) (. ?))


In [3]:
print(test_example)


(S (NP-SBJ (DT This)) (VP (VBZ is) (NP-PRD (NNP Japan))) (. ?))


The answer is **"1S  1VP  -1S"**.

We can specify each word like this.


$w_1$ : "This"
$w_2$ : "is"
$w_3$ : "Japan"
$w_4$ : "?"


First,
1. There is only one common ancestor betweem $w_1$ and $w_2$, and the nonterminal symbal at the lowest common ancestor is **S**.
2. There is two common ancestor betweem $w_2$ and $w_3$, and the nonterminal symbal at the lowest common ancestor is **VP**.
3. There is only one common ancestor betweem $w_3$ and $w_4$, and the nonterminal symbal at the lowest common ancestor is **S**.


A linearized tree in absolute scale, **"1S  2VP  1S"**.


Therefore, a linearized tree in relative scale, **"1S  1VP  -1S"**.



**Task 6: Now write a function that converts the tree to a linearized tree in an relative scale encoding (2P)**

_Your answer here._

In [4]:
# Function to get the common ancestors and their count using the relative scale
def relative_scale_encoding(tree):
    encoding = []  # 결과를 저장할 리스트 초기화
    leaves = tree.leaves()  # 트리의 잎 노드(단어) 가져오기
   
    temp = 0
    for i in range(len(leaves) - 1):
        w_i = leaves[i]   # 현재 단어
        w_i_1 = leaves[i + 1]  # 다음 단어
        
        common_ancestor = tree.treeposition_spanning_leaves(i, i + 2)  # 현재 단어와 다음 단어의 최소 공통 조상 가져오기
        common_ancestor_count = len(common_ancestor) + 1  # 최소 공통 조상의 깊이 계산


        # 상대적 척도 계산
        relative_scale = common_ancestor_count if i == 0 else common_ancestor_count - temp


        # 결과 리스트에 상대적 척도와 레이블 추가
        label = (relative_scale, tree[common_ancestor].label())
        encoding.append(label)


        temp = common_ancestor_count
   
    return encoding

# Get the common ancestors and their count for each pair of adjacent words
encoding = relative_scale_encoding(test_example)


# Print the transformed tree in the specified format
print("Original Tree:")
print(test_example)
print("\nTransformed Tree:")
for i, (n_i, c_i) in enumerate(encoding, start=1):
   print(f"  (w{i} ({n_i}, {c_i}))")


       


Original Tree:
(S (NP-SBJ (DT This)) (VP (VBZ is) (NP-PRD (NNP Japan))) (. ?))

Transformed Tree:
  (w1 (1, S))
  (w2 (1, VP))
  (w3 (-1, S))


Before we can train on our dataset, we have to transform the labels to class indices. For this purpose, we will use the LabelEncoder from sklearn (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

For each tree, the last token gets a dummy variable (e.g. ["-1", "DUMMY"])

In [5]:
!pip install -U scikit-learn

**Task 7: Create Class Labels. Fill in sentences, encodings and labels variables. (2P)**

In [6]:
# You might have to install sklearn first. Use the following command and execute it in another cell:
# !pip install -U scikit-learn
from sklearn.preprocessing import LabelEncoder

'''
Your Code here.
Transform the label pairs (e.g. (1, S)) to class labels. Fill in sentences, encodings and labels
'''

sentences = []
# sentences = Holds the sentence of the tree

encodings = []
# encodings = Holds the label pairs with a dummy variable at the end
# Hint: I would encode the labels with the dummy variable as ['-1', 'DUMMY']
# For our test example: [['1', 'S'], ['1', 'VP'], ['-1', 'S'], ['-1', 'DUMMY']]

class_labels = []
# labels = Holds the class indices, e.g. [768, 829, 165, 72] for one sentence (each token has one)
# Hint: I would transform each pair to a string by joining them with "_"
# E.g. '1_S', '1_VP', '-1_S', '-1_DUMMY', ...
# Collect all labels and fit your LabelEncoder on all labels (use .fit() method)
# Then transform each example with your LabelEncoder (use .transform() method)
# For our Test Example, I got: [768 829 165 72]

# 레이블 쌍을 클래스 레이블로 변환
# 예를 들어, (1, S)를 '1_S'로 변환
# 마지막으로 더미 레이블 '-1_DUMMY'를 추가
labels = [f"{n}{'' if n == 0 else '_'}{c}" for n, c in encoding] + ['-1_DUMMY']

# LabelEncoder를 초기화하고 모든 레이블에 적합화
label_encoder = LabelEncoder()
label_encoder.fit(labels)

# LabelEncoder를 사용하여 각 예제를 변환
class_labels = label_encoder.transform(labels)

# 결과를 출력
# sentences, encodings, labels 변수를 채움
sentences = [test_example.leaves()]  # 트리의 단어들을 가져와서 sentences 리스트에 추가
encodings = [[n, 'DUMMY'] if c == 'DUMMY' else [n, c] for n, c in encoding] + [[-1, 'DUMMY']]  # 레이블 쌍에 더미 레이블을 추가
class_labels_encoded = label_encoder.transform([f"{n}_{c}" for n, c in encoding] + ["-1_DUMMY"])  # 클래스 레이블에 더미 레이블을 추가

# 결과를 출력합니다.
print("\nSentences:", sentences)    
print("Encodings:", encodings)
print("Class Labels:", class_labels_encoded)




Sentences: [['This', 'is', 'Japan', '?']]
Encodings: [[1, 'S'], [1, 'VP'], [-1, 'S'], [-1, 'DUMMY']]
Class Labels: [2 3 1 0]


In [7]:
import numpy as np

# This were my values. 
# If you got different ones by using different methods, ignore this assert
assert sentences[439] == ['This', 'is', 'Japan', '?']
assert encodings[439] == [['1', 'S'], ['1', 'VP'], ['-1', 'S'], ['-1', 'DUMMY']]
assert np.array_equal(class_labels[439], np.array([768, 829, 165, 72]))


IndexError: list index out of range

We load the GloVe embeddings to subsequently use them for initializing our RNN. Use the same GloVe embeddings as we used in Homework 03. 

In [9]:
import numpy as np

# Load GloVe embeddings from the text file
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.strip().split()
            word = values[0]
            vector = np.array([float(val) for val in values[1:]])
            embeddings[word] = vector
    return embeddings

glove_file_path = 'glove.6B.50d.txt' 
glove_embeddings = load_glove_embeddings(glove_file_path)

# Create Embedding Index
word_to_index = {word: idx + 1 for idx, word in enumerate(glove_embeddings.keys())}
index_to_word = {idx + 1: word for idx, word in enumerate(glove_embeddings.keys())}

**Task 8: Build a PyTorch Dataset where the input consists of GloVe word indices, and utilize the class indices generated in the previous task as labels. (2P)**

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader


'''


Your code here.


Create PyTorch Dataset, where the input is the sentence indices of GloVe and
the created labels as class indices in the previous task. The dataset should be
created such that you can train an RNN later on the sentences (with GloVe vocab) and predicts the
class indices.


'''


class TreeDatasetGloVeIndexed(Dataset):
    def __init__(self, sentences, class_labels):
      self.sentences = sentences
      self.class_labels = class_labels


    def __len__(self):
        return len(self.sentences)


    def __getitem__(self, idx):
        input_sentence = self.sentences[idx]
        label = self.class_labels[idx]
        return torch.tensor(input_sentence), torch.tensor(label)


# We use a batch size of 1 only.
batch_size = 1


# Train dataloader
train_sentences = sentences[:3000]
train_labels = class_labels[:3000]
train_tree_dataset = TreeDatasetGloVeIndexed(train_sentences, train_labels)
train_dataloader = DataLoader(train_tree_dataset, batch_size=batch_size, shuffle=True)




# Dev/Val dataloader
dev_sentences = sentences[3000:3139]
dev_labels = class_labels[3000:3139]
dev_tree_dataset = TreeDatasetGloVeIndexed(dev_sentences, dev_labels)
dev_dataloader = DataLoader(dev_tree_dataset, batch_size=batch_size, shuffle=True)




# Test dataloader
test_sentences = sentences[3139:]
test_labels = class_labels[3139:]
test_tree_dataset = TreeDatasetGloVeIndexed(test_sentences, test_labels)
test_dataloader = DataLoader(test_tree_dataset, batch_size=batch_size, shuffle=True)




ValueError: num_samples should be a positive integer value, but got num_samples=0

**Task 9: Create a bidirectional RNN that initialized the embedding with the GloVe embedding! (2P)**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


'''


Your Code here.


Create a bidirectional RNN that initialized the embedding with the GloVe embedding!


Hint: Use nn.RNN() to create your RNN - Dont have to implement it yourself!


'''
# Define the bidirectional RNN model with GloVe embeddings initialization
class TreeBiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, glove_embeddings):
        super(TreeBiRNN, self).__init__()
       
        self.embedding = nn.Embedding.from_pretrained(glove_embeddings, freeze = True)
        self.rnn = nn.RNN(input_size, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)


    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])


        return output


# Define parameters
input_size = 50  # GloVe embeddings size
hidden_size = 64
output_size = 2


# Create the RNN model, loss function, and optimizer
model = TreeBiRNN(input_size, hidden_size, output_size, glove_embeddings)


**Task 10: Now create a training loop and train on your created dataloader! After 100 steps: Collect the training loss and the accuracy on the val/dev dataset . Plot the loss and accuracy curve at the end. (3P)**

_Hint: Similar to Homework 03._

In [ ]:
import torch.nn.functional as F

# Set Hyperparameters
num_epochs = 1 # You can change the epoch to get better performance
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Your Variables to fill!
collect_loss_after_steps = 100 
collected_losses = []
collected_val_accuarcy = []

'''
Your Code Here. This might take some minutes to run!

Create a training loop and train on your created dataloader!
After 100 steps: Collect the training loss and the accuracy on the val/dev dataset. 
Plot the loss and accuracy curve at the end. 

Feel free to test different RNN settings out to get the best performance! 
If one epoch is running too long, you can reduce the number of steps for training.

Hint: You have to transform the labels so that the dimensions are (batch_size x sequence_length x n_classes),
where the labels are one hot vectors: labels = F.one_hot(labels, num_classes=output_size).float()

'''

# train_dataloader holds the test data
# dev_dataloader holds the dev/val data



**Task 11: Now, test your bidirectional RNN! Use accuracy as a metric. (1P)**

_Hint: You can use the same function that you already implemented to calculate the accuracy for the dev/val set_

In [ ]:
# Testing loop
model.eval()  # Set the model to evaluation mode

'''
Your Code Here.

Test your bidirectional RNN! Use accuracy as a metric.

'''

# test_dataloader holds the test data
